In [1]:
from icap.database.icapdatabase import ICapDatabase
from icap.results.results import Results
from icap.peco.peco import PECORecipe, PECO, PECOInterval, PECODemand

import pandas as pd
import numpy as np
import datetime
import tempfile
import os

In [2]:
fp = 'icap/database/icapdatabase.json'
conn = ICapDatabase(fp).connect()

In [3]:
pdmd = PECODemand(conn)

In [4]:
pdmd.compute_icap().head()

(500, 9)
   Year      CPDate RateClass Strata  NARatio  NAAvg    RCLF  NSPLScale  \
0  2017  2017-07-19        AL    177      0.0    0.0     NaN   1.030154   
1  2017  2017-07-20        AL    177      0.0    0.0     NaN   1.030154   
2  2017  2017-07-21        AL    177      0.0    0.0     NaN   1.030154   
3  2016  2016-07-25        AL    177      0.0    0.0  1.1031   1.030154   
4  2016  2016-07-27        AL    177      0.0    0.0  1.1031   1.030154   

   PLCScaleFactor  
0        0.999553  
1        0.999553  
2        0.999553  
3        0.999553  
4        0.999553  
Record<id=0167137028, year=2017, rc=GS, strata=101>
Record<id=1073034019, year=2017, rc=GS, strata=101>
Record<id=1630700809, year=2017, rc=GS, strata=101>
Record<id=2114501608, year=2017, rc=SLE, strata=170>
Record<id=2149298011, year=2017, rc=HT, strata=151>
Record<id=4278001035, year=2017, rc=GS, strata=101>
Record<id=7159808024, year=2017, rc=HT, strata=151>
Record<id=7899056044, year=2017, rc=GS, strata=101>
Rec

,RunDate,ISO,Utility,PremiseId,Year,RateClass,Strata,MeterType,ICap
0,2018-08-25 14:02:05.120333,PJM,PECO,0167137028,2018,GS,101,DMD,227.194433
1,2018-08-25 14:02:05.120333,PJM,PECO,0167137028,2018,GS,101,DMD,227.194433
2,2018-08-25 14:02:05.120333,PJM,PECO,0167137028,2018,GS,101,DMD,227.194433
3,2018-08-25 14:02:05.120333,PJM,PECO,0167137028,2018,GS,101,DMD,227.194433
4,2018-08-25 14:02:05.120333,PJM,PECO,0167137028,2018,GS,101,DMD,227.194433


# PLC/ICAP Recipe

**Values**
1. Demand on CP Dates - Averaged
2. NA Ratio on CP Dates - Averaged
3. Corresponding PLC Scale Factor

$$ D * NA * RCLF * PLCScale = ICAP $$

In [5]:
# Values for the PLC/ICAP
records = pdmd.records_.copy()
util = pdmd.util_df_.copy()
sys = pdmd.sys_df_.copy()

In [6]:
# Transmission scaling factor for NITS
nspl_scale = sys[sys['ParameterId'] == 'TransLoadScaleFactor'].copy()
nspl_scale = nspl_scale.rename(columns={'ParameterValue': 'NSPLScale'}).drop(labels='ParameterId', axis=1)

In [7]:
# NA Ratio
# Average the NA ratio by [Year, RateClass, Strata]
na_ratio = util[util['ParameterId'] == 'NARatio'].copy()
na_ratio = na_ratio.rename(columns={'ParameterValue': 'NARatio'}).drop(labels='ParameterId', axis=1)
na_ratio['NAAvg'] = na_ratio.groupby(['Year', 'RateClass', 'Strata']).transform('mean') 

# Rate Class Loss Factor
rclf = util[util['ParameterId'] == 'RateClassLoss'].copy()
rclf = rclf.rename(columns={'ParameterValue': 'RCLF'}).drop(labels='ParameterId', axis=1)

# PLCScale
plcf = sys[sys['ParameterId'] == 'PLCScaleFactor'].copy()
plcf = plcf.rename(columns={'ParameterValue': 'PLCScaleFactor'}).drop(labels='ParameterId', axis=1)


# Merge the utility values; fill missing with NaN
# Forward fill fills by year and rateclass
util_min = pd.merge(na_ratio, rclf, on=['Year','CPDate', 'RateClass', 'Strata'], how='left'
                   ).fillna(method='ffill')

util_min = pd.merge(util_min, nspl_scale, on='Year', how='left')
util_min = pd.merge(util_min, plcf, on='Year', how='left')


In [8]:
# Average demand per premise per year
records['AvgDmd'] = records.groupby(['PremiseId', 'Year', 'RateClass', 'Strata'])['Demand'].transform('mean')
records['RecCount'] = records.groupby(['PremiseId', 'Year', 'RateClass', 'Strata'])['Demand'].transform('count')

In [9]:
util_min

,Year,CPDate,RateClass,Strata,NARatio,NAAvg,RCLF,NSPLScale,PLCScaleFactor
0,2017,2017-07-19,AL,177,0.00,0.00,NaN,1.030154,0.999553
1,2017,2017-07-20,AL,177,0.00,0.00,NaN,1.030154,0.999553
2,2017,2017-07-21,AL,177,0.00,0.00,NaN,1.030154,0.999553
3,2016,2016-07-25,AL,177,0.00,0.00,1.1031,1.030154,0.999553
4,2016,2016-07-27,AL,177,0.00,0.00,1.1031,1.030154,0.999553
5,2016,2016-08-10,AL,177,0.00,0.00,1.1031,1.030154,0.999553
6,2016,2016-08-11,AL,177,0.00,0.00,1.1031,1.030154,0.999553
7,2016,2016-08-12,AL,177,0.00,0.00,1.1031,1.030154,0.999553
8,2017,2017-06-12,AL,177,0.00,0.00,1.1031,1.030154,0.999553
9,2017,2017-06-13,AL,177,0.00,0.00,1.1031,1.030154,0.999553


In [119]:
records

,PremiseId,Year,StartDate,RateClass,Strata,Demand,AvgDmd,RecCount
0,0167137028,2017,2017-08-01,GS,101,231.0,231.000,2.0
1,0167137028,2017,2017-08-11,GS,101,231.0,231.000,2.0
2,1073034019,2017,2017-05-31,GS,101,145.9,149.775,4.0
3,1073034019,2017,2017-06-29,GS,101,149.8,149.775,4.0
4,1073034019,2017,2017-07-29,GS,101,153.6,149.775,4.0
5,1073034019,2017,2017-08-29,GS,101,149.8,149.775,4.0
6,1630700809,2017,2017-05-26,GS,101,180.0,177.000,4.0
7,1630700809,2017,2017-06-27,GS,101,183.0,177.000,4.0
8,1630700809,2017,2017-07-27,GS,101,201.0,177.000,4.0
9,1630700809,2017,2017-08-25,GS,101,144.0,177.000,4.0


In [290]:
class RecordWriter:
    def __init__(self, records=None):
        assert(records is not None)
        self.records = records
        
    def write(self, fp=None):
        if fp is None:
            fp = os.path.join(tempfile.gettempdir(), 'peco_demand_nits.csv')
            
        header = 'PREMISE,RATECLASS,STRATA,YEAR,RUNDATE,NA1,NA2,NA3,NA4,NA5,NAAVG,'\
                'DMD1,DMD2,DMD3,DMD4,DMDAVG,RCLF,PLCF,PLC,NSPLSCALE,NITS'
            
        with open(fp, 'w') as fout:
            fout.write(header + os.linesep)
            for record in self.records:
                fout.write(record.string_record + os.linesep)
        

In [291]:
# Record object to aggregate the information iteratively
class Record:
    def __init__(self, premise_id, year, rateclass, strata):
        self.premise_id = premise_id
        self.year = str(int(year) + 1)
        self.rateclass = rateclass
        self.strata = strata
        
        self.demand = None
        self.avg_demand = None
        
        self.na_ratio = None
        self.na_avg = None
        self.rclf = None
        self.nspl_scale = None
        self.plcf = None
        self.rundate = datetime.datetime.now()
        
        self.plc = None
        self.nits = None
        self.computed = False
        
    def __repr__(self):
        return 'Record<id={premise_id}, year={year}, rc={rateclass}, strata={strata}>'.format(**self.__dict__)
    
    def string_builder(self):
        assert(self.computed)
        return '{premise_id},{rateclass},{strata},{year},{rundate},{na_ratio},{na_avg},'\
                '{demand},{avg_demand},{rclf},{plcf},{plc},{nspl_scale},{nits}'.format(**self.__dict__)
    
    def compute(self):
        assert(self.avg_demand is not None)
        assert(self.na_avg is not None)
        assert(self.rclf is not None)
        assert(self.nspl_scale is not None)
        assert(self.plcf is not None)
        self.plc = self.avg_demand * self.na_avg * self.rclf * self.plcf
        self.nits = self.plc * self.nspl_scale
        
        self.computed = True
        self.string_record = self.string_builder()
        
        

In [292]:
# Group records
objs = list()
for k, v  in records.groupby(['PremiseId', 'Year', 'RateClass', 'Strata']):
    r = Record(*k)
    
    # Pad to ensure demand is always 4
    demand = v['Demand'].values
    demand = np.pad(demand, (0, 4 - len(demand)), 'constant', constant_values=np.NaN)
    r.demand = ','.join(str(x) for x in demand)
    r.avg_demand = v['AvgDmd'].values[0]
    objs.append(r)

In [293]:
for r in objs:
    # Filter params by record object
    c1 = util_min['Year'] == record.year
    c2 = util_min['Strata'] == record.strata
    c3 = util_min['RateClass'] == record.rateclass
    _df = util_min[c1 & c2 & c3]
    assert(_df.shape[0] == 5)
    
    # Pad to enusre ratios always have 5 values
    na_ratios = _df['NARatio'].values
    na_ratios = np.pad(na_ratios, (0, 5 - len(na_ratios)), 'constant', constant_values=np.NaN)
    r.na_ratio = ','.join(str(x) for x in na_ratios)
    r.na_avg = _df['NAAvg'].values[0]
    r.rclf = _df['RCLF'].values[0]
    r.nspl_scale = _df['NSPLScale'].values[0]
    r.plcf = _df['PLCScaleFactor'].values[0]
    
    r.compute()

In [294]:
rw = RecordWriter(objs)

In [295]:
rw.write()